In [1]:
# import packages
import yfinance as yf
import yahoo_fin.stock_info as si
import numpy_financial as np
from jproperties import Properties

Assumed Constants

In [2]:
TICKER = 'MSFT'

SnP_data = yf.download("^GSPC", period="10y", interval="1d")["Close"]

def calculate_cagr(initial_price, final_price, period):
  cagr = ((final_price / initial_price) ** (1 / period)) - 1
  return cagr

initial_price = SnP_data.iloc[0]
final_price = SnP_data.iloc[-1]
SnP_10Y_RETURN = calculate_cagr(initial_price, final_price, 10)

BOND_TICKER = '^TNX'


[*********************100%%**********************]  1 of 1 completed


Read Ticker Data

In [3]:
ticker = yf.Ticker(TICKER)
tnx = yf.Ticker(BOND_TICKER)

income_stmt_df = ticker.income_stmt
balance_sheet_df=ticker.balance_sheet

Return the latest non Nan Value

In [4]:
def get_latest_non_nan(series):
    first_non_nan_index = series.first_valid_index()
    return series[first_non_nan_index]

Cost of Debt Calculation

In [5]:
interest_expense = get_latest_non_nan(income_stmt_df.loc['Interest Expense'])
income_tax = get_latest_non_nan(income_stmt_df.loc['Tax Provision'])
income_before_tax = get_latest_non_nan(income_stmt_df.loc['Pretax Income'])
current_debt = get_latest_non_nan(balance_sheet_df.loc['Current Debt'])
long_term_debt = get_latest_non_nan(balance_sheet_df.loc['Long Term Debt'])
cost_of_debt = interest_expense / (current_debt + long_term_debt)
effective_tax_rate = income_tax / income_before_tax
cost_of_debt_after_tax = cost_of_debt * (1-effective_tax_rate)
cost_of_debt_after_tax

0.03375533279028093

Cost of Equity Calculation

In [6]:
risk_free_rate = tnx.info['regularMarketDayHigh']/100
beta = ticker.info['beta']
cost_of_equity = risk_free_rate + (beta * (SnP_10Y_RETURN - risk_free_rate))
cost_of_equity

0.09399435042137105

Weight of Debt and Equity

In [7]:
total_debt = current_debt + long_term_debt
market_cap = ticker.info['marketCap']
total= total_debt + market_cap

weight_of_debt = total_debt / total
weight_of_equity = market_cap / total

WACC Calculation

In [8]:
wacc = (weight_of_debt * cost_of_debt_after_tax) + (weight_of_equity * cost_of_equity)
'{:.2%}'.format(round(wacc,4)) + ' for ticker: ' + TICKER

'9.30% for ticker: MSFT'